# Diffusion Analyser

Welcome to the ***Diffusion Analyser*** which enables the analysis of ion diffusion in a solid-electrolyte from an AIMD trajectory and determine the uncertaintiy in the calculated diffusion coefficient.

This notebook shows the capabilities of the code and allows the user to look into how different factors play affect the diffusion coefficient in order to get a better estimate of the uncertaintiy associated with the calculated diffusion coefficient.

In [ ]:
# import required modules
# please ensure all the modules listed on the README are installed in order for this code to run correctly.
import matplotlib.pyplot as plt
import numpy as np
import ase
import ase.io
from ase.visualize import view
from ase import units
from ase.io.trajectory import Trajectory
from matplotlib.ticker import AutoMinorLocator

The **DiffusionAnalsysis** class, which is based of the ASE DiffusionCoefficient class calculates the mean squared displacement (MSD) and fits the MSD vs 6t curve to obtain the diffucion coefficient, using either linear least squares or bayesian linear regression. 

This can be done for all species present in the structure, a chosen species, or a provided list of atom indices. The trajectory can also be split into several segements and the diffusion coefficient is then calculated separately for each individual segement and the total trajectory. The standard error of the fit is also provided in each case.

In [ ]:
# stop execution class in case of invalid input parameters
class StopExecution(Exception):
    def _render_tracebeck_(self):
        pass

class DiffusionAnalysis:
    
    def __init__(self, trajectory, timestep, species='Li', atom_indices=None, unwrap=True,
                 no_of_segments=1, fitting_algo='lstsq', MSD=None, MSD2=None, MSD3=None):
        """
        trajectory: trajectory of md run read in from .traj file
        timestep: timestep between each image of the trajectory in fs time units, i.e. dt*img_steps 
        species: species for which MSD is to be calculated, can be either 'all', or one of the species
            in Li-argyrodite, i.e. 'Li', 'P', 'S', or 'Cl'; default is 'Li'
        atom_indices: indices of atoms whose MSD is to be calculated explicitly
        no_of_segments = get MSD for given number of segments for statistical analysis. Total length of 
            the trajectory needs to be divisible by the number of segements to obtain segements of equal length.
        fitting_algo: algorithm used to fit diffusion coefficient - 
            can be 'lstsq' for linear least squares or 'bayesian' for Bayesian linear regression 
        MSD - array of mean squared displacements from previous calculation of the same trajectory
        MSD2 - array of mean sqauared displacement from previous calculation of a different trajectory
        MSD3 - array of mean sqauared displacement from previous calculation of a third trajectory
        MSD, MSD2, and MSD3 need to be supplied for bayesian linear regression
        """
        # main code taken from https://wiki.fysik.dtu.dk/ase/_modules/ase/md/analysis.html#DiffusionCoefficient
        self.timestep = timestep
        self.species = species
        self.MSD = MSD
        self.MSD2 = MSD2
        self.MSD3 = MSD3
        self.fitting_algo = fitting_algo
        if fitting_algo == 'bayesian':
            assert type(self.MSD2) == np.ndarray
            assert type(self.MSD3) == np.ndarray
        
        # unwrap trajectory if required 
        if unwrap == True:
            self.trajectory = self.unwrap_coords(trajectory)
        else:
            self.trajcetory = trajectory
        # get relevant atom indices from user specified input
        if self.species == 'all'and atom_indices == None:
            self.atom_indices = [i for i in range(len(self.trajectory[0]))]
        if self.species != 'all' and atom_indices == None:
            self.atom_indices = np.argwhere(self.trajectory[0].symbols == species)[:,0]
        if atom_indices != None:
            self.atom_indices = atom_indices
        # determine the types and numbers of atoms of interest
        self.types_of_atoms = sorted(set(self.trajectory[0].symbols[self.atom_indices]))
        self.no_of_atoms = [self.trajectory[0].get_chemical_symbols().count(symbol) for symbol in self.types_of_atoms]
        # determine length of segments and check whether it is an integer
        self.no_of_segments = no_of_segments
        self.len_segments = len(self.trajectory) // self.no_of_segments
        if len(self.trajectory) % self.no_of_segments != 0:
            print('Total length of the trajectory must be divisible by the number of segments')
        assert len(self.trajectory) % self.no_of_segments == 0
        
        
        # get MSD either by calculating it new or from supplied MSD and fit the slopes, standard error 
        # in the slopes and the intercepts for given number of segments
        if self.no_of_segments == 1:
            if type(self.MSD) == np.ndarray:
                self._MSD = self.MSD
            else:
                self._MSD = self.calculate_MSD()
            self._seg_slopes, self._seg_stderrs, self._seg_intercepts = self.fit_data()
            self._slopes = self._seg_slopes[:,0]
            self._intercepts = self._seg_intercepts[:,0]
            self._stderrs = self._seg_stderrs[:,0]
        else:
            # fit slope, i.e. D and intercept for every segments
            # due to every segment being fitted individually and the problem that len_segments needs to be an integer
            # and is this rounded down, the mean of the individual diffusion coefficients for each segments is NOT the
            # same as the diffusion coefficient fitted for the while trajectory!!!
            if type(self.MSD) == np.ndarray:
                self._MSD = self.MSD
            else:
                self._MSD = self.calculate_MSD()
            self._seg_MSD = self.get_seg_MSD(self._MSD)
            if type(self.MSD2) == np.ndarray:
                self.seg_MSD2 = self.get_seg_MSD(self.MSD2)
            if type(self.MSD3) == np.ndarray:
                self.seg_MSD3 = self.get_seg_MSD(self.MSD3)
            self._seg_slopes, self._seg_stderrs, self._seg_intercepts = self.fit_data()
            # also do a fit for the whole trajectory
            self.no_of_segments = 1
            self.len_segments = len(self.trajectory) // self.no_of_segments
            seg_slopes, seg_stderrs, seg_intercepts = self.fit_data()
            self._slopes = seg_slopes[:,0]
            self._intercepts = seg_intercepts[:,0]
            self._stderrs = seg_stderrs[:,0]
            self.no_of_segments = no_of_segments
     
    def initialise_arrays(self):
        """
        creates arrays to hold data
        """
        self.timesteps = np.linspace(0,len(self.trajectory)*self.timestep,len(self.trajectory)+1)
        self.xyz_ensemble_average = np.zeros((self.no_of_segments, len(self.types_of_atoms), 3, self.len_segments))
        self.slopes_list = np.zeros((len(self.types_of_atoms), self.no_of_segments, 3))
        self.intercepts_list = np.zeros((len(self.types_of_atoms), self.no_of_segments, 3))
        self.res_list = np.zeros((len(self.types_of_atoms), self.no_of_segments, 3))
        self.stderrs_list = np.zeros((len(self.types_of_atoms), self.no_of_segments, 3))
        self.seg_slopes = np.zeros((len(self.types_of_atoms), self.no_of_segments))
        self.seg_intercepts = np.zeros((len(self.types_of_atoms), self.no_of_segments))
        self.seg_residuals = np.zeros((len(self.types_of_atoms), self.no_of_segments))
        self.seg_stderrs = np.zeros((len(self.types_of_atoms), self.no_of_segments))
    
    
    def calculate_MSD(self):
        """
        caclulates the mean squared displacement
        """
        self.initialise_arrays()
        for segment_no in range(self.no_of_segments):
            # set up segments with correct length
            start = segment_no*self.len_segments
            end = start + self.len_segments
            seg = self.trajectory[start:end]
            for image_no in range(0, len(seg)):
                xyz_disp = np.zeros((len(self.types_of_atoms),3))
                # calculate displacement of each atom in atom_indices and sum together for each type of atom
                for atom_no in self.atom_indices:
                    atom_type_ix = self.types_of_atoms.index(seg[image_no].symbols[atom_no])
                    xyz_disp[atom_type_ix] += np.square(seg[image_no].positions[atom_no] - 
                                                        seg[0].positions[atom_no])
                # calculate average of the data from xyz_disp for each type of atom
                for atom_type_ix in range(len(self.types_of_atoms)):
                    denominator = (2*self.no_of_atoms[atom_type_ix])
                    for xyz in range(3):
                        self.xyz_ensemble_average[segment_no][atom_type_ix][xyz][image_no] = (xyz_disp[atom_type_ix][xyz]/denominator)
        return self.xyz_ensemble_average
    
    def get_seg_MSD(self, MSD):
        """
        extracts the MSD for each segment from the full MSD and writes to an array of the correct format
        """
        self.initialise_arrays()
        for segment_no in range(self.no_of_segments):
            # set up segments with correct length
            start = segment_no*self.len_segments
            end = start + self.len_segments
            for atom_type_ix in range(len(self.types_of_atoms)):
                for xyz in range(3):
                    self.xyz_ensemble_average[segment_no][atom_type_ix][xyz] = MSD[0][atom_type_ix][xyz][start:end]
        return self.xyz_ensemble_average
        
    
    def fit_data(self):
        """
        fits the data either using linear least squares or bayesian linear regression
        """
        # fit data to determine slope, i.e diffusion coefficient, and intercept of y = mx+c like MSD vs timesteps graph
        self.initialise_arrays()
        if self.no_of_segments == 1:
            self.xyz_ensemble_average = self._MSD
        else:
            self.xyz_ensemble_average = self._seg_MSD
        for segment_no in range(self.no_of_segments):
            # set up segments with correct length
            start = segment_no*self.len_segments
            end = start + self.len_segments
            seg = self.trajectory[start:end]
            # if fitting algo is lstsq, fitted using fit_lstsq based on numpy.linalg.lstsq
            if self.fitting_algo=='lstsq':
                for atom_type_ix in range(len(self.types_of_atoms)):
                    # get slope, intercept and residuals for each atom type and each direction (x,y, and z)
                    self.slopes_list[atom_type_ix][segment_no], self.intercepts_list[atom_type_ix][segment_no], \
                    self.res_list[atom_type_ix][segment_no] = self.fit_lstsq(
                        self.timesteps[start:end], self.xyz_ensemble_average[segment_no][atom_type_ix])
                    # get the mean of all three directions for each atom type
                    self.seg_slopes[atom_type_ix][segment_no] = np.mean(self.slopes_list[atom_type_ix][segment_no])
                    self.seg_intercepts[atom_type_ix][segment_no] = np.mean(self.intercepts_list[atom_type_ix][segment_no])
                    self.seg_residuals[atom_type_ix][segment_no] = np.mean(self.res_list[atom_type_ix][segment_no])
                    # calculate standard error in slope
                    seg_n = len(seg)
                    seg_denom = (seg_n*np.sum((self.timesteps[start:end])**2)-np.sum(self.timesteps[start:end]) ** 2) * (seg_n - 2)
                    self.seg_stderrs[atom_type_ix][segment_no] = np.sqrt(seg_n*self.seg_residuals[atom_type_ix][segment_no] / seg_denom) 
            # if fitting algo is bayesian, fitted using fit_bayesian based on bayesian linear regression 
            if self.fitting_algo=='bayesian':
                y_sig = self.calc_y_sig()
                for atom_type_ix in range(len(self.types_of_atoms)):
                    # get slope, intercept and residuals for each atom type and each direction (x,y, and z)
                    self.slopes_list[atom_type_ix][segment_no], self.intercepts_list[atom_type_ix][segment_no], \
                    self.stderrs_list[atom_type_ix][segment_no] = self.fit_bayesian(
                        self.timesteps[start:end], self.xyz_ensemble_average[segment_no][atom_type_ix], y_sig[segment_no][atom_type_ix])
                    # get the mean of all three directions for each atom type
                    self.seg_slopes[atom_type_ix][segment_no] = np.mean(self.slopes_list[atom_type_ix][segment_no])
                    self.seg_intercepts[atom_type_ix][segment_no] = np.mean(self.intercepts_list[atom_type_ix][segment_no])
                    # calculate standard error in slope
                    self.seg_stderrs[atom_type_ix][segment_no] = np.mean(self.stderrs_list[atom_type_ix][segment_no])
        return self.seg_slopes, self.seg_stderrs, self.seg_intercepts


### Helper functions ### 

    # main code taken from Bora Karasulu's in-house code 'vasp_plot_RMS_traj.py'and adjusted
    def unwrap_coords(self, trajectory):
        """
        unwrapps coordinates if input coordinates are wrapped
        """
        # get no_of_atom and original scaled positions
        noAtoms = len(trajectory[0])
        init_pos=trajectory[0].get_scaled_positions()
        ppos= init_pos
        # set up place-holder array for shifts
        shifts=np.zeros((noAtoms,3))
        # convert first frame of trajectory to an atoms project and save in new trajectory file
        # new trajectory file is needed since the new scaled positions are not saved for trajectory objects
        atoms = trajectory[0]
        traj_new = Trajectory('unwrapped_traj.traj', 'w')
        traj_new.write(atoms)
        # calculate unwrapped shift of each atom (in x, y and z-direction) compared to its previous steps
        for st in range(0,len(trajectory)):
            atoms=trajectory[st]
            if st!=0:
                cpos=atoms.get_scaled_positions(wrap=False)
                for at in range(noAtoms):
                    for j in range(3):
                        diff=cpos[at][j]-ppos[at][j] 
                        # calculate correct shift if preiodic boundaries are crossed
                        if np.fabs(diff) >0.5: 
                            shifts[at][j]+=(diff - np.sign(diff))
                        else: shifts[at][j]+=diff
                # set the unwrapped scaled positions for the repective atom object and save it to new trajectory
                atoms.set_scaled_positions(init_pos+shifts)
                traj_new.write(atoms)
                ppos=cpos
        traj_new.close()
        # read in new trajectory 
        trajectory_unwrapped = Trajectory('unwrapped_traj.traj')
        return trajectory_unwrapped
    
    def fit_lstsq(self, x, y):
        # create space-holder arrays
        slopes = np.zeros(3)
        intercepts = np.zeros(3)
        residuals = np.zeros(3)
        # add second dimension to x data
        x_edited = np.vstack([np.array(x), np.ones(len(x))]).T
        for xyz in range(3):
            (slopes[xyz], intercepts[xyz]), residuals[xyz], rank, s = np.linalg.lstsq(
                x_edited,y[xyz], rcond=None)
        return slopes, intercepts, residuals

    
    def calc_y_sig(self):
        # concatenate MSD arrays into one array
        if self.no_of_segments == 1:
            displacements = np.array([self.MSD, self.MSD2, self.MSD3])
        else:
            displacements = np.array([self._seg_MSD, self.seg_MSD2, self.seg_MSD3])
        
        # set up arrays to hold data
        mean_displacements = np.zeros((self.no_of_segments, len(self.types_of_atoms), 3, displacements.shape[4]))
        y_sig = np.zeros((self.no_of_segments, len(self.types_of_atoms), 3, displacements.shape[4]))
        
        # calculate mean displacements from the number of supplied MSD arrays
        mean_displacements = np.mean(displacements, axis=0)
        # calculate the standard derivation for each trajectory point
        for segment_no in range(self.no_of_segments):
            for atom_type_ix in range(len(self.types_of_atoms)):
                for xyz in range(3):
                    for i in range(displacements.shape[4]):
                        y_sig[segment_no][atom_type_ix][xyz][i] = np.sqrt(np.sum((displacements[:, segment_no, atom_type_ix, xyz, i]-
                                                                              mean_displacements[segment_no, atom_type_ix, xyz, i])**2/len(displacements)))
        return y_sig
    
    # taken from https://github.com/libAtoms/matscipy/blob/master/matscipy/elasticity.py#L802
    def fit_bayesian(self, x, y, y_sig):
        '''
        Use analytical blr to solve y = m * x + c given x, y, and assumed errors

        y_sig : float | array
            Assumed variance of each observation.
        '''
        # arrays to hold gradients, intercepts and standard error for all 3 xyz directions
        slopes = np.zeros(3)
        intercepts = np.zeros(3)
        stderrs = np.zeros(3)
        
        # Design Matrix1
        X = np.array([x[1:], np.ones(x[1:].shape)]).T
        # Prior Precision Matrix
        Gamma = np.diag([1/ (10**(-4))**2, 1/1**2])
        for xyz in range(3):
            # Likelihood Precision
            Lambda = np.eye(y[xyz][1:].shape[0]) / y_sig[xyz][1:]**2
            # Solve for posterior
            posterior_variance= X.T @ Lambda @ X + Gamma
            # 2x2, so direct inverse not unstable 
            posterior_precision = np.linalg.inv(posterior_variance)
            slopes[xyz], intercepts[xyz] = posterior_precision @ X.T @ Lambda @ y[xyz][1:]
            stderrs[xyz] = np.sqrt(posterior_precision[0, 0]/len(X)) # Only care about gradient variance
        return slopes, intercepts, stderrs
    
    ### Separate functions to print and plot the results ###
    
    def print_output(self):
        """
        print calcualted diffusion coefficient and std for each species and segments
        """
        if self.no_of_segments !=1:
            print('segments')
            for atom_type_ix in range(len(self.types_of_atoms)):
                print('---')
                print(r'Species: {}'.format(self.types_of_atoms[atom_type_ix]))
                print('---')
                for segment_no in range(self.no_of_segments):
                    print(r'Segment   {}:         Diffusion Coefficient = {:.4e} Å^2/fs; Std. Errs. = {:.4e} Å^2;'.format
                          (segment_no, self._seg_slopes[atom_type_ix][segment_no], self._seg_stderrs[atom_type_ix][segment_no]))
        print('---')
        # print calcualated diffusion coefficient and std for each species for whole trajectory
        for atom_type_ix in range(len(self.types_of_atoms)):
            print('Mean Diffusion Coefficient (X, Y and Z) : {} = {:.4e} $\AA ^2$ Å^2/fs; Std. Errs. = {:.4e} Å^2/fs'.format
                  (self.types_of_atoms[atom_type_ix], self._slopes[atom_type_ix], self._stderrs[atom_type_ix]))
        print('---')
        
    
    def plot_MSD(self, species = 'Li'):
        """
        plots the MSD and the fitted curve
        species: select which species to plot the results for from the ones the MSD has been calculated for
        """
        x = self.timesteps[0:len(self.trajectory)]
        fig, ax = plt.subplots(figsize=(9,6))
        # plot MSD and fitted curve for all species for which the MSD has been calculated
        if species == 'all':
            for atom_type_ix in range(len(self.types_of_atoms)):
                # caclulate mean MSD for each atom type
                mean = np.mean(self.xyz_ensemble_average[0][atom_type_ix], axis=0)
                # create line of best fit from slopes and intercepts
                y = self._slopes[atom_type_ix]*x + self._intercepts[atom_type_ix]
                # plot MSD and fitted curve
                ax.plot(x, mean, label='{} MSD curve'.format(self.types_of_atoms[atom_type_ix]))
                ax.plot(x, y, label='{} fitted curve'.format(self.types_of_atoms[atom_type_ix]))
        # plot MSD and fitted curve for selected species
        if species == 'Cl' or species =='Li' or species == 'P' or species == 'S':
            # get the ix in types_of_atoms array for given species
            ix = np.argwhere(np.array(self.types_of_atoms) == species)[0,0]
            # calculate mean MSD for given type of atom
            mean = np.mean(self.xyz_ensemble_average[0][ix], axis=0)
            y = self._slopes[ix]*x + self._intercepts[ix]
            ax.plot(x, mean, label='{} MSD curve'.format(self.types_of_atoms[ix]))
            ax.plot(x, y, label='{} fitted curve'.format(self.types_of_atoms[ix]))
        # customise plot
        ax.set_xlabel('timesteps (fs)', size=14)
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.set_ylabel(r'MSD $(\AA^2)$', size=14)
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.set_title('MSD vs. time', y=1.025, size=18)
        ax.tick_params(axis='both', labelsize=12)
        plt.legend()    
        return plt.show()

Three trajectories from VASP AIMD runs, each of 40ps length, of the solid-electrolyte Li-argyrodite (Li6PS5Cl) are provided.

In [ ]:
# read in trajectory
trajectory_1 = Trajectory('./trajectories/trajectory1.traj')
trajectory_2 = Trajectory('./trajectories/trajectory2.traj')
trajectory_3 = Trajectory('./trajectories/trajectory3.traj')

The time step used in the trajectories was 2 fs, which is safed in the variable _dt_.
The total length of each trajectory is 40 ps, with each originally having 20000 steps. Due to file size limits, the trajectories given above contain every 5th step, i.e 4000 steps. 

How many steps of the trajectory are used can be controlled with the 'img_steps' keyword. This is one of the parameters that can be changed to investigates how it affects the resulting diffusion coefficient.

In [ ]:
# time step used in vasp trajectories
dt = 2
# adjust trajectory to not include every single image to reduce computational cost
# every nth step of original trajectory to be written
img_steps = 5

# provided trajectories already only contain every 5th step, so img_steps/5 gives the number of steps needed
trajectory1 = trajectory_1
trajectory2 = trajectory_2
trajectory3 = trajectory_3

The mean squared displacements (MSDs) for each trajectory are also already provided, since calculating these is the most time, and thus computational cost intensive step.

In [ ]:
MSD_1 = np.loadtxt('./MSD_arrays/MSD1.csv', delimiter=",")
MSD1 = MSD_1[None,None,:,::5]
MSD_2 = np.loadtxt('./MSD_arrays/MSD2.csv', delimiter=",")
MSD2 = MSD_2[None,None,:,::5]
MSD_3 = np.loadtxt('./MSD_arrays/MSD3.csv', delimiter=",")
MSD3 = MSD_3[None,None,:,::5]

### Run the diffusion analyser

The parameters that can need to be provided are:
1. trajectory : trajectory of AIMD run as .traj file
2. timestep: timestep between each image of the trajectory in ASE time units, i.e. dt*img_steps

The parameters that can be provided are:
1. species: species for which MSD is to be calculated, can either be 'all' or in this case one of the species in Li-argyrdite, i.e. Cl, Li, S or P. Default = Li
2. atom_indices: indices of atoms whose MSD is to be calculated explicitly; default is None, meaning that the atom indices are determined based on the species provided
3. fitting_algo: algorithm used to fit diffusion coefficient - can be lstsq for linear least squares (default) or bayesian for Bayesian linear
4. no_of_segments: get MSD for given number of segments for statistical analysis, default is 1, i.e. the whole trajectory. The total length of the trajectory needs to be divisible by the number of segements to obtain segements of equal length
5. MSD: if the MSD has been calculated previously for the required atom indices, it can be given as input to prevent having to re-calculate it every time. Required as input for bayesian linear regression. Default is 'None'
6. MSD2: previously calculated MSD of a different trajectory. Required as input for bayesian linear regression. Default is 'None'
7. MSD3: previously calculated MSD of a third trajectory. Required as input for bayesian linear regression. Default is 'None'

In [ ]:
# calculate diffusion coefficient using the DiffusionAnalysis class implemented above
C1 = DiffusionAnalysis(trajectory1, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=1, MSD=MSD1)
C2 = DiffusionAnalysis(trajectory2, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=1, MSD=MSD2)
C3 = DiffusionAnalysis(trajectory3, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=1, MSD=MSD3)

The individual outputs of intereste, i.e. the slopes, intercepts and standard errrors of the fit can be accessed in the following way.

In [ ]:
# Access the relevant outputs
# trajectory 1 
slopes1 = C1._slopes
stderrs1 = C1._stderrs
intercepts1 = C1._intercepts

#trajectory 2
slopes2 = C2._slopes
stderrs2 = C2._stderrs
intercepts2 = C2._intercepts

#trajectory 3
slopes3 = C3._slopes
stderrs3 = C3._stderrs
intercepts3 = C3._intercepts

The diffusion coefficient for each species with the standard error of the fitting can be printed like this:

In [ ]:
C1.print_output()
C2.print_output()
C3.print_output()

The MSD and the fitted curve can also be plotted using the provided plotting function 'plot_MSD'. It takes the extra argument 'species' in case only one of the species of the ones for which the diffusion coefficient was calculated is to be plotted. E.g. D has been calculated for all species but only the Li MSD is to be plotted.

Only plots the whole trajectory, not different segments. 

In [ ]:
C1.plot_MSD(species = 'Li')
C2.plot_MSD(species = 'Li')
C3.plot_MSD(species = 'Li')

The trajectory can be split into several segments and the diffusion coefficient is then calculated for each segement and the total trajectory. 

In [ ]:
C1_seg_2 = DiffusionAnalysis(trajectory1, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=2, MSD=MSD1)
C2_seg_2 = DiffusionAnalysis(trajectory2, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=2, MSD=MSD2)
C3_seg_2 = DiffusionAnalysis(trajectory3, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=2, MSD=MSD3)

C1_seg_4 = DiffusionAnalysis(trajectory1, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=4, MSD=MSD1)
C2_seg_4 = DiffusionAnalysis(trajectory2, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=4, MSD=MSD2)
C3_seg_4 = DiffusionAnalysis(trajectory3, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=4, MSD=MSD3)

C1_seg_8 = DiffusionAnalysis(trajectory1, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=8, MSD=MSD1)
C2_seg_8 = DiffusionAnalysis(trajectory2, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=8, MSD=MSD2)
C3_seg_8 = DiffusionAnalysis(trajectory3, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=8, MSD=MSD3)

C1_seg_16 = DiffusionAnalysis(trajectory1, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=16, MSD=MSD1)
C2_seg_16 = DiffusionAnalysis(trajectory2, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=16, MSD=MSD2)
C3_seg_16 = DiffusionAnalysis(trajectory3, dt*img_steps, 'Li', fitting_algo='lstsq', no_of_segments=16, MSD=MSD3)

The results can be printed as before.

In [ ]:
C1_seg_2.print_output()
C1_seg_4.print_output()
C1_seg_8.print_output()
C1_seg_16.print_output()

C2_seg_2.print_output()
C2_seg_4.print_output()
C2_seg_8.print_output()
C2_seg_16.print_output()

C3_seg_2.print_output()
C3_seg_4.print_output()
C3_seg_8.print_output()
C3_seg_16.print_output()

It can be seen that that there is an euqilibration period. Cutting the first 3 segments of the 16 segments, i.e. the first 750 ps, gives a better approxiamtion of the true diffusion coefficient.

In [ ]:
cut_trajectory1 = trajectory1[750:]
cut_trajectory2 = trajectory2[750:]
cut_trajectory3 = trajectory3[750:]

cut_MSD1_ = MSD1[0,0,:,750:]
cut_MSD1 = cut_MSD1_[None,None,:,:]
cut_MSD2_ = MSD2[0,0,:,750:]
cut_MSD2 = cut_MSD2_[None,None,:,:]
cut_MSD3_ = MSD3[0,0,:,750:]
cut_MSD3 = cut_MSD3_[None,None,:,:]

In [ ]:
C1_cut = DiffusionAnalysis(cut_trajectory1, dt*img_steps, fitting_algo = 'lstsq', MSD=cut_MSD1)
C2_cut = DiffusionAnalysis(cut_trajectory2, dt*img_steps, fitting_algo = 'lstsq', MSD=cut_MSD2)
C3_cut = DiffusionAnalysis(cut_trajectory3, dt*img_steps, fitting_algo = 'lstsq', MSD=cut_MSD3)

In [ ]:
C1_cut.print_output()
C2_cut.print_output()
C3_cut.print_output()

In [ ]:
C1_cut.plot_MSD()
C2_cut.plot_MSD()
C3_cut.plot_MSD()

To fit the graph using bayesian linear regression, the MSDs of all three trajectories need to be supplied to obtain a standard derivation for every data point.

In [ ]:
C1_bays = DiffusionAnalysis(cut_trajectory1, dt*img_steps, fitting_algo='bayesian', MSD=cut_MSD1, MSD2=cut_MSD2, MSD3=cut_MSD3)
C2_bays = DiffusionAnalysis(cut_trajectory2, dt*img_steps, fitting_algo='bayesian', MSD=cut_MSD2, MSD2=cut_MSD1, MSD3=cut_MSD3)
C3_bays = DiffusionAnalysis(cut_trajectory3, dt*img_steps, fitting_algo='bayesian', MSD=cut_MSD3, MSD2=cut_MSD1, MSD3=cut_MSD2)

In [ ]:
C1_bays.print_output()
C2_bays.print_output()
C3_bays.print_output()

In [ ]:
C1_bays.plot_MSD()
C2_bays.plot_MSD()
C3_bays.plot_MSD()

The standard error provided is the standard error of the gradient in the respective linear regression fit.
Calculating the diffusion coefficient for a larger number of segements for each of the three trajectories allows for good statistical anaysis of the standard deviation of the Diffusion coefficient. Since the total length of the trajectory needs to be divisible by the number of segments to ensure equally long segments, 5 and 13 segements were chosen here.

Again, the MSDs of all three trajectories have to be provided.

In [ ]:
cut_C1_seg5_bays = DiffusionAnalysis(cut_trajectory1, dt*img_steps, 'Li', fitting_algo='bayesian', 
                                   no_of_segments=5, MSD=cut_MSD1, MSD2=cut_MSD2, MSD3=cut_MSD3)
cut_C1_seg13_bays = DiffusionAnalysis(cut_trajectory1, dt*img_steps, 'Li', fitting_algo='bayesian', 
                                   no_of_segments=13, MSD=cut_MSD1, MSD2=cut_MSD2, MSD3=cut_MSD3)
cut_C2_seg5_bays = DiffusionAnalysis(cut_trajectory2, dt*img_steps, 'Li', fitting_algo='bayesian', 
                                   no_of_segments=5, MSD=cut_MSD2, MSD2=cut_MSD1, MSD3=cut_MSD3)
cut_C2_seg13_bays = DiffusionAnalysis(cut_trajectory2, dt*img_steps, 'Li', fitting_algo='bayesian', 
                                   no_of_segments=13, MSD=cut_MSD2, MSD2=cut_MSD1, MSD3=cut_MSD3)
cut_C3_seg5_bays = DiffusionAnalysis(cut_trajectory3, dt*img_steps, 'Li', fitting_algo='bayesian', 
                                   no_of_segments=5, MSD=cut_MSD3, MSD2=cut_MSD1, MSD3=cut_MSD2)                                      
cut_C3_seg13_bays = DiffusionAnalysis(cut_trajectory3, dt*img_steps, 'Li', fitting_algo='bayesian', 
                                   no_of_segments=13, MSD=cut_MSD3, MSD2=cut_MSD1, MSD3=cut_MSD2)

In [ ]:
cut_C1_seg5_bays.print_output()
cut_C1_seg13_bays.print_output()
cut_C2_seg5_bays.print_output()
cut_C2_seg13_bays.print_output()
cut_C3_seg5_bays.print_output()
cut_C3_seg13_bays.print_output()

In [ ]:
# save all the slopes in one array to calculate grand mean and pooled variance later
seg_slopes = []
seg_slopes1_5 = cut_C1_seg5_bays._seg_slopes[0]
seg_slopes1_13 = cut_C1_seg13_bays._seg_slopes[0]
seg_slopes2_5 = cut_C2_seg5_bays._seg_slopes[0]
seg_slopes2_13 = cut_C2_seg13_bays._seg_slopes[0]
seg_slopes3_5 = cut_C3_seg5_bays._seg_slopes[0]
seg_slopes3_13 = cut_C3_seg13_bays._seg_slopes[0]

seg_slopes.append(seg_slopes1_5)
seg_slopes.append(seg_slopes1_13)
seg_slopes.append(seg_slopes2_5)
seg_slopes.append(seg_slopes2_13)
seg_slopes.append(seg_slopes3_5)
seg_slopes.append(seg_slopes3_13)

Obtain the segmental diffusion coefficients (5 and 13 segments) for trajectories with every 25th and 50th step written to include possible variation in the calculated standard deviation.

In [ ]:
trajectories = [trajectory_1, trajectory_2, trajectory_3]
Img_steps = [50, 25]

for i in range(len(Img_steps)):
    img_steps = Img_steps[i]/5
    for x in range(3):
        trajectory = trajectories[x][0:-1:int(img_steps)]
        cut_trajectory = trajectory[int(750/img_steps):]

        MSD1_ = MSD1[:,:,:,::int(img_steps)]
        MSD2_ = MSD2[:,:,:,::int(img_steps)]
        MSD3_ = MSD3[:,:,:,::int(img_steps)]
        cut_MSD1_ = MSD1_[0,0,:,int(750/img_steps):]
        cut_MSD1 = cut_MSD1_[None,None,:,:]
        cut_MSD2_ = MSD2_[0,0,:,int(750/img_steps):]
        cut_MSD2 = cut_MSD2_[None,None,:,:]
        cut_MSD3_ = MSD3_[0,0,:,int(750/img_steps):]
        cut_MSD3 = cut_MSD3_[None,None,:,:]

        C_seg5_bays = DiffusionAnalysis(cut_trajectory, dt*img_steps, 'Li', fitting_algo='bayesian', 
                                       no_of_segments=5, MSD=cut_MSD1, MSD2=cut_MSD2, MSD3=cut_MSD3)
        seg_slopes_5 = C_seg5_bays._seg_slopes[0]
        C_seg13_bays = DiffusionAnalysis(cut_trajectory, dt*img_steps, 'Li', fitting_algo='bayesian', 
                                       no_of_segments=13, MSD=cut_MSD1, MSD2=cut_MSD2, MSD3=cut_MSD3)
        seg_slopes_13 = C_seg13_bays._seg_slopes[0]
        seg_slopes.append(seg_slopes_5)
        seg_slopes.append(seg_slopes_13)

Calculate the grand mean and pooled standard deviation from all the obtained data.

In [ ]:
Means = []
N = []
N_s = []
for i in range (len(seg_slopes)):
    m = np.mean(seg_slopes[i])
    Means.append(m)
    s = np.std(seg_slopes[i])
    n = len(seg_slopes[i])
    n_s = (n-1)*s**2
    N.append(n)
    N_s.append(n_s)
mean = np.sum(Means)/len(seg_slopes)
std = np.sqrt(np.sum(N_s)/(np.sum(N)-len(seg_slopes)))

print('The mean diffusion coefficient is {:.4e} Å^2/fs with a standard deviation of {:.4e}'.format(mean, std))